In [ ]:
!pip install langchain
!pip install openai
!pip install pandas
!pip install langchain_community # Install the missing langchain_community package
!pip install langchain_openai

import os
import openai
import pandas as pd
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set your OpenAI API key
openai.api_key = 'sk-proj-...'
from langchain_openai import ChatOpenAI
# Initialize OpenAI LLM
llm = OpenAI(api_key=openai.api_key)
chat_model = ChatOpenAI(api_key=openai.api_key,model="gpt-4o",temperature=0.0)
# Define a simple prompt template
prompt_template = PromptTemplate(
    input_variables=["data"],
    template='''Assistant, I need you to analyze the transcript between a doctor and a patient and extract specific information. Your task is to give the Reason for Call::...</c>, Age:...</a>, Gender:...</g>, Symptoms:...</s>,Disease:...</d>, Diagnosis:...</di>, and prescription:...</p> based on the conversation.

    Here is the transcript:

    {data}

Please extract the necessary information and including all the required details. Remember to accurately identify and always in this format Reason for Call:..</c>, Age:...</a>, Gender:...</g>, Symptoms:...</s>, Disease:...</d>,Diagnosis:..</di>, and Prescription:..</p>. If not interpretable, not specified, not mentioned for example in age, Age:NA</a> so on.
    Text:
    ''')


# Create an LLMChain with the LLM and the prompt template
chain = LLMChain(llm=chat_model, prompt=prompt_template)
a={}
# Generate a response using the LLMChain
def ds(data):
  response = chain.run(data=data)

# Print the response
  return(response)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
def remtag(Age,s,f):
    if s in Age:
      Age=Age.replace(s,'')
    else:
      Age=Age.replace(f,'')
    return Age

In [ ]:
import re
# Function to load data from a .txt file
def load_data_from_txt(file_path):
    with open(file_path, 'r') as file:
        data = file.read()
    return data

In [ ]:
call=[]
age=[]
gender=[]
disease=[]
symptoms=[]
diagnosis=[]
prescription=[]
for file_path in os.listdir('/content/drive/MyDrive/Clean Transcripts/'):
# Load data from the .txt file
  file = load_data_from_txt('/content/drive/MyDrive/Clean Transcripts/'+file_path)
  response=ds(file)
  print(response)
  if(response[:3]=='```'):
    response=response.replace('``` \n','')
    response=response.replace(' \n```','')
  if(response[:2]=='**'):
    response=response.replace('**','')
  if(response[-2:]=='**'):
    response=response.replace('**','')
  #print(response)
  Reason=re.findall(r'Reason for Call.*(?:<c>|<\/c>)', response)[0]
  Reason=remtag(Reason,'</c>','<c>').replace('Reason for Call:','')
  call.append(Reason)
  Age=re.findall(r'Age.*(?:<a>|<\/a>)', response)[0]
  Age=remtag(Age,'</a>','<a>').replace('Age:','')
  age.append(Age)
  Gender=re.findall(r'Gender.*(?:<g>|<\/g>)', response)[0]
  Gender=remtag(Gender,'</g>','<g>').replace('Gender:','')
  gender.append(Gender)
  Symptoms=re.findall(r'Symptoms.*(?:<\/s>|<s>)', response)[0]
  Symptoms=remtag(Symptoms,'</s>','<s>').replace('Symptoms:','')
  symptoms.append(Symptoms)
  Disease=re.findall(r'Disease.*(?:<\/d>|<d>)', response)[0]
  Disease=remtag(Disease,'</d>','<d>').replace('Disease:','')
  disease.append(Disease)
  Diagnosis=re.findall(r'Diagnosis.*(?:<\/di>|<di>)', response)[0]
  Diagnosis=remtag(Diagnosis,'</di>','<di>').replace('Diagnosis:','')
  diagnosis.append(Diagnosis)
  Prescription=re.findall(r'Prescription.*(?:<\/p>|<p>)', response)[0]
  Prescription=remtag(Prescription,'</p>','<p>').replace('Prescription:','')
  prescription.append(Prescription)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Reason for Call: Severe abdominal pain in the upper right area</c>, Age: NA</a>, Gender: NA</g>, Symptoms: Severe sharp pain in the upper right abdomen, nausea, decreased energy levels, reduced appetite</s>, Disease: Possible gallbladder disease</d>, Diagnosis: Abdominal pain, likely related to gallbladder issues</di>, Prescription: Not specified</p>
Reason for Call: Suspected bladder infection</c>, Age: 30</a>, Gender: Female</g>, Symptoms: Burning sensation during urination, frequent urination, feeling hot, tiredness, cloudy urine</s>, Disease: Overactive bladder, possible urinary tract infection (UTI)</d>, Diagnosis: Suspected urinary tract infection (UTI)</di>, Prescription: Investigations to be conducted, possible antibiotics (pending confirmation from physician)</p>.
Reason for Call: Intense pain in right shoulder following a rugby incident</c>, Age: NA</a>, Gender: NA</g>, Symptoms: Intense, constant sharp pain in right shoulder, inability to move shoulder, numbness above should

In [ ]:
a={"Reason for Call":call,"Age":age,"Gender":gender,"Symptoms":symptoms,"Disease":disease,"Diagnosis":diagnosis,"Prescription":prescription}
print(a)

{'Reason for Call': [' Severe abdominal pain in the upper right area', ' Suspected bladder infection', ' Intense pain in right shoulder following a rugby incident', ' Back pain', ' Daughter experiencing pain and limping in the right leg', ' Diarrhea', ' Pain in the right knee and calf after a fall three days ago', ' Back pain and stiffness', ' Sharp pain in chest, fear of having a heart attack', ' Chest pain', ' Pain in left elbow for a couple of weeks, worsening over the last three weeks.', ' Right knee pain, swelling, and warmth', ' Pain in the knee impacting running', ' Severe back pain and associated symptoms', ' Pain in the right buttock area radiating down the right leg', ' Chest pain', ' Low back pain', ' Abdominal pain', " Stomach ache, vomiting, and fever in the patient's son", ' Pain in the right elbow', " Feeling breathless and it's getting worse", ' Back pain for the past month', ' Knee pain after playing basketball and hearing a pop in the left knee', ' Numbness, tingling,

In [ ]:
a=pd.DataFrame(a)
a.to_csv('data.csv')

In [ ]:
a.head()

,Reason for Call,Age,Gender,Symptoms,Disease,Diagnosis,Prescription
0,Severe abdominal pain in the upper right area,NA,NA,"Severe sharp pain in the upper right abdomen,...",Possible gallbladder disease,"Abdominal pain, likely related to gallbladder...",Not specified
1,Suspected bladder infection,30,Female,"Burning sensation during urination, frequent ...","Overactive bladder, possible urinary tract in...",Suspected urinary tract infection (UTI),"Investigations to be conducted, possible anti..."
2,Intense pain in right shoulder following a ru...,NA,NA,"Intense, constant sharp pain in right shoulde...",Shoulder dislocation,Shoulder dislocation confirmed by physical ex...,X-ray to confirm dislocation and check for fr...
3,Back pain,NA,NA,"Lower back pain for two weeks, sharp pain, wo...",Mechanical back pain,"Based on history and physical exam, likely me...","Modified activity at work, short-term bed res..."
4,Daughter experiencing pain and limping in the...,5,Female,"Limping, pain in the right leg and hip, diffi...","Transient synovitis, Septic arthritis, Legg-C...","Transient synovitis (primary), Septic arthrit...",Anti-inflammatories such as ibuprofen
